# PDF to text

In [6]:
import PyPDF2

def extract_text(filepath, text=[]):
    # Open the PDF file in read-binary mode
    pdf_reader = PyPDF2.PdfReader(filepath)

    # Get the number of pages in the PDF file
    num_pages = len(pdf_reader.pages)

    # Loop over each page and extract the text
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        text += page.extract_text().split("\n")


    return text

pdf_text = extract_text("/Users/senthil/Desktop/Senthil/my_testing/Project_Creation/testing_briefs/purchase_orders20230803 (1).pdf")


# for txt in pdf_text:
#     print(txt)

In [3]:
file_path = "/Users/senthil/Desktop/Senthil/my_testing/Project_Creation/testing_briefs/purchase_orders 00185079pdf.docx"

In [4]:
import docx, re
import tabulate as tab
from datetime import datetime
import pandas as pd

def extract_docx_to_list(file_path):
    # Open the Word document
    doc = docx.Document(file_path)

    # Initialize an empty list to store extracted text
    docx_para_list = []

    # Iterate through each element in the document
    for element in doc.element.xpath('.//*'):
        # Check if the element is a paragraph
        if element.tag.endswith('}p'):
            # Skip paragraphs that are inside table cells
            if element.getparent().tag.endswith('}tc'):
                continue
            # Get all text elements in the paragraph
            text_elements = element.xpath('.//w:t')
            if text_elements:
                # Concatenate the text from all text elements
                text = ''.join([text_element.text for text_element in text_elements])
                docx_para_list.append(text)

        # Check if the element is a table
        elif element.tag.endswith('}tbl'):
            # Iterate through each row in the table
            for row in element.xpath('.//w:tr'):
                # Iterate through each cell in the row
                for cell in row.xpath('.//w:tc'):
                    cell_text = []
                    # Get all paragraphs in the cell
                    for paragraph in cell.xpath('.//w:p'):
                        runs = paragraph.xpath('.//w:r')
                        cell_text_by_paragraph = ""
                        # Concatenate the text from all runs in each paragraph
                        for run in runs:
                            run_text = run.xpath('.//w:t')
                            if run_text:
                                cell_text_by_paragraph += run_text[0].text
                        cell_text.append(cell_text_by_paragraph)

                    # Check if the cell contains a dropdown list
                    result_val = cell.xpath('.//w:r/w:fldChar/w:ffData/w:ddList/w:result/@w:val')
                    if result_val:
                        selected_value = cell.xpath('.//w:ffData/w:ddList/w:listEntry/@w:val')
                        if selected_value:
                            # Iterate through each paragraph in the cell and add the selected value for each dropdown to the list
                            for i, cell_text_by_paragraph in enumerate(cell_text):
                                if i < len(result_val) and i < len(selected_value):
                                    docx_para_list.append(f"{cell_text_by_paragraph} {selected_value[int(result_val[i])]}")
                                else:
                                    docx_para_list.append(cell_text_by_paragraph)
                        else:
                            # Iterate through each paragraph in the cell and add its text to the list
                            for cell_text_by_paragraph in cell_text:
                                docx_para_list.append(cell_text_by_paragraph)
                    else:
                        # Iterate through each paragraph in the cell and add its text to the list
                        for cell_text_by_paragraph in cell_text:
                            docx_para_list.append(cell_text_by_paragraph)

    return docx_para_list
# EEP

text = extract_docx_to_list(file_path)

# text = text +pdf_text


def find_text_with_regexes(regex_dict, text):
    # Create an empty dictionary to hold the results
    results = {}
    my_text = '\t'.join(text)
    print(my_text)
    # Loop through each label and regex in the dictionary
    for label, regex in regex_dict.items():
        # Use regex to search for the pattern in the text
        # print(f"Searching for label '{label}' with regex '{regex}'")
        match = re.search(regex, my_text)

        # If a match is found, add it to the results dictionary with the current label as the key
        if match:
            # print(f"Found match '{match.group(2)}'")
            results[label] = match.group(2).strip()
        # else:
        #     print(f"{label} not found")

    # Return the results dictionary
    return results


regex_dict = {"book_title":"(?i)(book\s+title|title(?<!Subtitle))(?::|,|-|–)?\s?(.+?(?=\t))",
"subtitle":"(?i)(book\ssub\s?title|sub\s?title)(?::|,|-|–)?\s?(.+?(?=\t))",
"series":"(?i)(Series)(?::|,|-|–)?\s?(.+?(?=\t))",
"author":"(Name:\s?Role:\s?Email:|Author\/Eds|Author(?:s?)(?:\(s\))?|contact\sauthor(?:s?)(?:\(s\))|(?<!Anthem\s)Editor(?:s)?(?:\(s\))?|Contributor(?:s?)(?:\(s\)))(?::|,|-|–)?\s?(.+?(?=\t))",
"author_mailid":"(?i)((?:Email(?::|,|-|–)?(?:\t).+?\t?(?:Editor|Author)\t?)|\(email|Author Contact.+?)[:]?\s?([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)",
"isbn":"(?i)(ISBN)\s?(?<!E-ISBN)(?:[:,-–])?(.+?)(?<=\t)",
"hardback_isbn":"(?i)(ISBN\s\(hardback\)|ISBN\s\(hard|cloth\)|hardback)\s?(?::|,|-|–)?\s?(.+?(?=\t))",
"paperback_isbn":"(?i)(ISBN\s\(paperback\)|ISBN\s\(paper\)|paperback|NIPPOD)\s?(?::|,|-|–)?\s?(.+?(?=\t))",
"epub_isbn":"(?i)(ePub\sebook|ePub|ISBN\s\(elec\.\)):\s?(?::|,|-|–)?\s?(.+?(?=\t))",
"epub2_isbn":"(?i)(XML)\s?(?::|,|-|–)\s?(.+?(?=\t))",
"ebook_master":"(?i)(ePDF|Web\sPDF|(?<!ePub\s)ebook|(?<!print-ready\stext\s)PDF|E-ISBN)\s?(?::|,|-|–)\s?(.+?(?=\t))",
"isbn_opt":"(?i)(\s)?(.+?(?=\t))\((ISBN(?:\s?))\)",
"hardback_isbn_opt":"(?i)(\s)?(.+?(?=\t))\((Hardback)\)",
"paperback_isbn_opt":"(?i)(\s)?(.+?(?=\t))\((paperback)\)",
"epub_isbn_opt":"(?i)(\s)?(.+?(?=\t))\((ePub)\)",
"ebook_master_opt":"(?i)(\s)?(.+?(?=\t))\((Web\sPDF)\)",
"po_number":"(?i)(PO\sNumber|MUP\skeycode|Purchase\sOrder(?:\sNumber)?|P\.O\.\s\#)\s?(?::|,|-|–)?\s?([\w\d]+?(?=\t)(he\/him)?)",
"page_count":"(?i)(Estimated\sprint\sextent|Estimated\spage\sextent|Est\.\sfinal\spp\scount|Extent)\s?(?::|,|-|–)?\s?(.+?(?=\t))",
"project_startDate":"(?i)(Handover\sdate|start\sdate|date(?<!Requested\sfinals\sdate)(?!\sdue|s\sare styled|\sreferences used in|\sreferences).\))\s?(?::|,|-|–)?\s?(.+?(?=\t))",
"project_endDate":"(?i)(Target\ssend-to-print\sdate\sFinal\sdue|Requested\sfinals\sdate|Target\ssend-to-print\sdate|Schedule:\sReturn\smanuscript\sby|Final\sPDF\s\+\sPOD\sfiles|Printer\sDate|Final\sPDF\sdue|End date)\s?(?::|,|-|–)?\s?(?:late\s|spring|Publication date)?(.+?(?=\t))",
"production_editor":"(Bloomsbury\sProduction\scontact|Anthem\sEditor|Production\sEditor|^From|Bloomsbury Production Editor|PE)\s?(?::|,|-|–)(.+?(?=\t))(,\s?Rowman\s\&\sLittlefield)?",
"project_manager":"(?i)(To)\s?(?:[:,-–])?(?:[:,-–]?\s+)?(.+?(?=\t))(,\s?Deanta Global)",
"text_design":"(?i)(Text\sdesign\stemplate|Text\sdesign|^template|Design\stemplate|Layout\sstyle.+?X|Design)\s?(?::|,|-|–)?\s?(.+?(?=\t))",
"trim_size":"(?i)(TPS|Page\strim|Format|Trim\ssize|Trim)\s?(?:[:,-–]?)(?:\s{1,})?(\d{1,3}\s?(?:\.?\d{1,3}?)?\s?x\s?\d{1,3}\.?(?:\.?\d{1,3}?)?\s?)\s?(?:mm|in)?(?:.+?)(?=\t)",
"Indexer":"(?i)(Index[:]\sYes\s☒\sNo\s☐\s+Compiled\sby[:]\s+)(.+?(?=\t))",
"productioneditor_mailid":"(?i)(, Email(?::|,|-|–)?)(.+?(?=\t))",
"binding":"(?i)(Price\s+\d+\s[-]\s)(.+?(?=\t))",
"published_date":"(?i)(Target\spublication\sdate)\s?(?::|,|-|–)?\s??(.+?(?=\t))"}


results = find_text_with_regexes(regex_dict, text)

# cleanup the extracted texts

def metadata_cleanup(results):    

    # removing ":" from isbn
    pattern = r"\d+\.\d+ - First Printing|£\d+\.\d+ GBP|[-\s:]|\(cloth\)|\(paper\)|\(elec\.\)|\bN(?:\/)?A\b|\(delayed\)|\(NIPPOD\)|ISBN|Paperback|£\d+\.\d+GBP"

    # List of keys for which the ISBN values need to be cleaned
    isbn_keys_to_clean = ["isbn", "hardback_isbn", "paperback_isbn", "epub_isbn", "epub2_isbn", "ebook_master", "isbn_opt", "hardback_isbn_opt", "paperback_isbn_opt", "epub_isbn_opt", "ebook_master_opt"]
    # Clean ISBN values for each key in 'keys_to_clean'
    for key in isbn_keys_to_clean:
        if key in results and results[key] is not None:
            print(f"harback ISBN: {results[key]}")
            results[key] = re.sub(pattern, "", results[key])


    # replacing the "isbn" with "hardback_isbn"
    try:
        if not results["isbn"]:
            _default_hardback_isbn = results.get("hardback_isbn", "")
            results["isbn"]= _default_hardback_isbn
            if not results["isbn"]:
                _default_paperback_isbn = results.get("paperback_isbn", "")
                results["isbn"]= _default_paperback_isbn
    except:
        print("ISBN is not found")


    # removing prefix from "author"
    authors_pattern = "(?i)Dr(\.)?|\bEdited by\b|,\sRowman\s\&\sLittlefield\s(?:Publishing\sGroup)?|\(R\&L template\)|\bN(?:\/)?A\b|\bProfessor\b|Handbook format|\[(?:under|over)\s\d+K\]|\(she\/her\)"

    person_keys_to_clean = ["author", "production_editor", "project_manager","text_design","subtitle","series"]
    for key in person_keys_to_clean:
        if key in results and results[key] is not None:
            results[key] = re.sub(authors_pattern, "", results[key])

    return results

updated_metadata = metadata_cleanup(results)



# Convert the dictionary to a list of lists (rows) for tabulate
data = [[key, value] for key, value in updated_metadata.items()]

# Print the dictionary as a table
print(tab.tabulate(data, headers=["Key", "Metadata"], tablefmt="fancy_grid"))


Purchase Order	Trade Typesetting (match design)	Bloomsbury	Publishing	Bill To:	Bloomsbury Publishing (UK)	50 Bedford Square, London, WC1B	3DP	Telephone: (0)20 7631 5600	Fax: (0)20 7631 5800	Email:	bloomsburyaccounts@bloomsbury.com	To:  De anta - Trade - UK owne d	51 Clontarf Road	Dublin 3	Contact Simon Eaton, Email: Simon.Eaton@bloomsbury.com	PO Numbe r00185079	Date 07/08/2023	TitleWild Woman	AuthorPhilippa Forrester	Other Suppliers	Printer/Binder: CPI Trade	Edition I SBNPr ic e	ISBN:9781399400879	Common Specification	Product TypeBook	Extent272	Format216 x 135mm - trimmed. 216mm (81/ ") (h) x 135mm (55/  ") (w)	2 16	Preparation & TextText Print: 1x1. Print From: Disk.	Page ComplexityMedium	Images printing on text paperImages printing on text paper - TBC.	Foil Blocking - StandardFoil Blocking - Standard - TBC (using OTTO Wolves finishes as guidance for now).	Notes	Please supply Trade Typesetting services for this title, according to the workflow and terms defined in our Trade Typesettin